<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/[NOTEBOOK_ID])

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community

Transform your AI ideas into reality through hands-on projects and expert mentorship.

[Start Your Journey](https://www.buildfastwithai.com/genai-course)

---

# Gemini 3 Pro - Advanced Features

**Created by:** @BuildFastWithAI  
**Model:** Google Gemini 3 Pro  
**Last Updated:** November 2025

This notebook covers advanced features including:
- Streaming responses
- Advanced function calling
- Structured output and JSON mode
- Context management
- Caching and optimization
- Error handling and retries

In [ ]:
!pip install -q google-generativeai pydantic tenacity

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import json
from typing import List, Optional
from pydantic import BaseModel, Field
from tenacity import retry, stop_after_attempt, wait_exponential

# Configure API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## 1. Streaming Responses

In [ ]:
# Basic streaming
model = genai.GenerativeModel('gemini-3-pro')

response = model.generate_content(
    "Write a detailed explanation of neural networks.",
    stream=True
)

print("Streaming response:")
for chunk in response:
    print(chunk.text, end='', flush=True)
print("\n\n✅ Streaming complete")

In [ ]:
# Streaming with function calling
tools = [
    {
        "function_declarations": [
            {
                "name": "search_web",
                "description": "Search the web for information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "Search query"}
                    },
                    "required": ["query"]
                }
            }
        ]
    }
]

model_with_tools = genai.GenerativeModel('gemini-3-pro', tools=tools)
chat = model_with_tools.start_chat()

response = chat.send_message(
    "Search for the latest AI news and summarize it.",
    stream=True
)

for chunk in response:
    if chunk.text:
        print(chunk.text, end='', flush=True)

## 2. Advanced Function Calling

In [ ]:
# Parallel function calls
def get_stock_price(symbol: str) -> dict:
    """Get stock price for a symbol."""
    # Simulated data
    prices = {"AAPL": 175.50, "GOOGL": 142.30, "MSFT": 380.20}
    return {"symbol": symbol, "price": prices.get(symbol, 0)}

def get_company_info(symbol: str) -> dict:
    """Get company information."""
    info = {
        "AAPL": {"name": "Apple Inc.", "sector": "Technology"},
        "GOOGL": {"name": "Alphabet Inc.", "sector": "Technology"},
        "MSFT": {"name": "Microsoft Corp.", "sector": "Technology"}
    }
    return info.get(symbol, {})

# Function schemas
finance_tools = [
    {
        "function_declarations": [
            {
                "name": "get_stock_price",
                "description": "Get current stock price",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {"type": "string", "description": "Stock symbol"}
                    },
                    "required": ["symbol"]
                }
            },
            {
                "name": "get_company_info",
                "description": "Get company information",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "symbol": {"type": "string", "description": "Stock symbol"}
                    },
                    "required": ["symbol"]
                }
            }
        ]
    }
]

model = genai.GenerativeModel('gemini-3-pro', tools=finance_tools)
chat = model.start_chat()

response = chat.send_message(
    "Get me the stock price and company info for AAPL and GOOGL"
)
print(response.text)

## 3. Structured Output with Pydantic

In [ ]:
# Define structured output schema
class ProductReview(BaseModel):
    product_name: str = Field(description="Name of the product")
    rating: int = Field(description="Rating from 1-5", ge=1, le=5)
    pros: List[str] = Field(description="List of pros")
    cons: List[str] = Field(description="List of cons")
    summary: str = Field(description="Brief summary")
    would_recommend: bool = Field(description="Whether to recommend")

# Create prompt for structured output
review_text = """
I bought the XYZ Laptop last month. It has amazing battery life and a beautiful display.
The performance is great for coding and video editing. However, it's quite expensive
and the trackpad could be better. Overall, I'm happy with my purchase.
"""

prompt = f"""
Analyze this product review and return ONLY valid JSON matching this schema:
{ProductReview.model_json_schema()}

Review: {review_text}

Return JSON only, no additional text.
"""

model = genai.GenerativeModel(
    'gemini-3-pro',
    generation_config={"response_mime_type": "application/json"}
)

response = model.generate_content(prompt)
review_data = json.loads(response.text)
print(json.dumps(review_data, indent=2))

# Validate with Pydantic
validated_review = ProductReview(**review_data)
print(f"\n✅ Validated: {validated_review.product_name}")

## 4. Advanced Prompting Techniques

In [ ]:
# Few-shot learning
few_shot_prompt = """
Classify the sentiment of customer feedback.

Examples:
Input: "The product is amazing! Best purchase ever."
Output: {"sentiment": "positive", "confidence": 0.95}

Input: "Terrible quality. Very disappointed."
Output: {"sentiment": "negative", "confidence": 0.90}

Input: "It's okay, nothing special."
Output: {"sentiment": "neutral", "confidence": 0.75}

Now classify:
Input: "Good value for money, but shipping was slow."
Output:
"""

model = genai.GenerativeModel(
    'gemini-3-pro',
    generation_config={"response_mime_type": "application/json"}
)
response = model.generate_content(few_shot_prompt)
print(response.text)

In [ ]:
# Chain-of-thought prompting
cot_prompt = """
Solve this problem step by step:

A store has 150 items. On Monday, they sold 30% of their inventory.
On Tuesday, they received a shipment of 50 new items.
On Wednesday, they sold 25 items.
How many items do they have now?

Think through this step by step:
"""

model = genai.GenerativeModel('gemini-3-pro')
response = model.generate_content(cot_prompt)
print(response.text)

## 5. Context Management

In [ ]:
# Conversation with context window management
class ManagedChat:
    def __init__(self, max_history: int = 10):
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.chat = self.model.start_chat()
        self.max_history = max_history
        self.messages = []
    
    def send_message(self, message: str) -> str:
        # Add user message
        self.messages.append({"role": "user", "content": message})
        
        # Truncate if needed
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]
        
        # Send message
        response = self.chat.send_message(message)
        
        # Add assistant message
        self.messages.append({"role": "assistant", "content": response.text})
        
        return response.text
    
    def get_context_size(self) -> int:
        return len(self.messages)

# Test managed chat
chat = ManagedChat(max_history=6)
print(chat.send_message("Hello! I'm learning about AI."))
print(f"\nContext size: {chat.get_context_size()}")

## 6. Batch Processing & Rate Limiting

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

class RateLimitedBatchProcessor:
    def __init__(self, requests_per_minute: int = 60):
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.delay = 60.0 / requests_per_minute
        self.last_request = 0
    
    def process_with_rate_limit(self, prompt: str) -> str:
        # Rate limiting
        now = time.time()
        time_since_last = now - self.last_request
        if time_since_last < self.delay:
            time.sleep(self.delay - time_since_last)
        
        self.last_request = time.time()
        response = self.model.generate_content(prompt)
        return response.text
    
    def batch_process(self, prompts: List[str], max_workers: int = 3) -> List[str]:
        results = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_prompt = {
                executor.submit(self.process_with_rate_limit, prompt): prompt 
                for prompt in prompts
            }
            
            for future in as_completed(future_to_prompt):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    print(f"Error: {e}")
                    results.append(None)
        
        return results

# Test batch processing
processor = RateLimitedBatchProcessor(requests_per_minute=30)
prompts = [
    "Explain quantum computing in one sentence.",
    "What is machine learning?",
    "Define artificial intelligence."
]

print("Processing batch...")
results = processor.batch_process(prompts, max_workers=2)
for i, result in enumerate(results, 1):
    print(f"\nResult {i}: {result[:100]}...")

## 7. Caching & Optimization

In [ ]:
from functools import lru_cache
import hashlib

class CachedModel:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
        self.cache = {}
    
    def _hash_prompt(self, prompt: str) -> str:
        return hashlib.md5(prompt.encode()).hexdigest()
    
    def generate(self, prompt: str, use_cache: bool = True) -> str:
        if use_cache:
            cache_key = self._hash_prompt(prompt)
            if cache_key in self.cache:
                print("✅ Cache hit!")
                return self.cache[cache_key]
        
        print("🔄 Generating new response...")
        response = self.model.generate_content(prompt)
        
        if use_cache:
            self.cache[cache_key] = response.text
        
        return response.text

# Test caching
cached_model = CachedModel()
prompt = "What is the capital of France?"

print("First call:")
result1 = cached_model.generate(prompt)
print(result1[:100])

print("\nSecond call (should use cache):")
result2 = cached_model.generate(prompt)
print(result2[:100])

## 8. Error Handling & Retries

In [ ]:
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from google.api_core import exceptions

class RobustModel:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-3-pro')
    
    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(multiplier=1, min=2, max=10),
        retry=retry_if_exception_type((exceptions.ResourceExhausted, exceptions.ServiceUnavailable))
    )
    def generate_with_retry(self, prompt: str) -> str:
        """Generate content with automatic retry on transient errors."""
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"⚠️ Error occurred: {type(e).__name__}")
            raise
    
    def generate_safe(self, prompt: str, default: str = "Error occurred") -> str:
        """Generate with fallback on error."""
        try:
            return self.generate_with_retry(prompt)
        except Exception as e:
            print(f"❌ All retries failed: {e}")
            return default

# Test error handling
robust_model = RobustModel()
result = robust_model.generate_safe("Explain machine learning.")
print(result[:200])

## 9. Key Takeaways

✅ **Advanced Capabilities:**
- Streaming for real-time responses
- Parallel function calling
- Structured JSON output with validation
- Sophisticated prompting techniques

🔧 **Production Best Practices:**
- Implement rate limiting
- Use caching for repeated queries
- Add retry logic with exponential backoff
- Manage context window efficiently

📊 **Optimization Tips:**
- Batch similar requests
- Use streaming for long outputs
- Cache common responses
- Monitor token usage

🔗 **Resources:**
- [Advanced Features Docs](https://ai.google.dev/)
- Follow [@BuildFastWithAI](https://twitter.com/BuildFastWithAI)